In [1]:
source("path/preprocess/utils.R")
setwd("path")
library(RColorBrewer)

parser <- ArgumentParser()
parser$add_argument("--task", type = "character", default = "dcm_hcm")
parser$add_argument("--method", type = "character", default = "midas_embed")
parser$add_argument("--experiment", type = "character", default = "inputs")
parser$add_argument("--model", type = "character", default = "default")
parser$add_argument("--init_model", type = "character", default = "")
o <- parser$parse_known_args()[[1]]

Attaching SeuratObject

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: 'S4Vectors'


The following objects are masked from 'package:base':

    I, expand

In [2]:
obj <- Seurat::Read10X('path/data/raw/dcm_hcm/')
obj

'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").

  [[ suppressing 32 column names 'TTCTTCCGTTCAACGT-1-0', 'CATCCACCATCTAACG-1-0', 'ACCCAAACAGCTAACT-1-0' ... ]]

  [[ suppressing 32 column names 'TTCTTCCGTTCAACGT-1-0', 'CATCCACCATCTAACG-1-0', 'ACCCAAACAGCTAACT-1-0' ... ]]

  [[ suppressing 32 column names 'TTCTTCCGTTCAACGT-1-0', 'CATCCACCATCTAACG-1-0', 'ACCCAAACAGCTAACT-1-0' ... ]]



36601 x 592689 sparse Matrix of class "dgCMatrix"
                                                                                             
MIR1302-2HG   .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
FAM138A       .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
OR4F5         .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
AL627309.1    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
AL627309.3    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
AL627309.2    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
AL627309.5    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  1  .  .  .  .  .  .  .
AL627309.4    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
AP006222.2    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
AL732372.1

In [3]:
obj <- Seurat::CreateSeuratObject(obj)
obj

An object of class Seurat 
36601 features across 592689 samples within 1 assay 
Active assay: RNA (36601 features, 0 variable features)

In [6]:
subset_name_list <- list()
subset_names <- c("P1290","P1300","P1304","P1358","P1371","P1422","P1425","P1430","P1437","P1447","P1462","P1472","P1479","P1504","P1508","P1510","P1515","P1516",
"P1539","P1540","P1547","P1549","P1558","P1561","P1582","P1600","P1602","P1603","P1606","P1610","P1617","P1622","P1630","P1631",
"P1678","P1685","P1702","P1707","P1718","P1722","P1726","P1735")
pp_dir <- 'path/data/processed/dcm_hcm/'

cell_name_list <- list()
for(i in c(1:42)){
    subset_name <- subset_names[[i]]
    cell_name_list[[subset_name]] <- read.csv(pj(pp_dir, paste0("subset_", i-1),
        "cell_names.csv"), header = T)[, 2]
}
cell_name <- do.call("c", unname(cell_name_list))

In [8]:
obj <- obj[, gsub("^.*_", "", cell_name)]
obj

An object of class Seurat 
36601 features across 523369 samples within 1 assay 
Active assay: RNA (36601 features, 0 variable features)

In [9]:
for(i in c(1:42)){
    subset_name <- subset_names[[i]]
    subset_name_list[[subset_name]] <- rep(subset_name, length(cell_name_list[[subset_name]]))
}
obj@meta.data$batch <- factor(x = do.call("c", unname(subset_name_list)), levels = subset_names)

In [17]:
label_list <- list()
label_paths <- list()
for(i in c(1:42)){
    label_paths[[i]] <- pj('/opt/data/private/xx/code/MIRACLE/data/raw/dcm_hcm/',subset_names[[i]],'/label_seurat/l2.csv')
    label_list[[subset_names[[i]]]] <- read.csv(label_paths[[i]], header = T)[, 2]
}

In [18]:
obj@meta.data$l2 <- do.call("c", unname(label_list))

In [24]:
FindVariableFeatures(obj)

An object of class Seurat 
36601 features across 523369 samples within 1 assay 
Active assay: RNA (36601 features, 2000 variable features)

In [ ]:
obj <- ScaleData(obj)
obj <- RunPCA(obj, features = VariableFeatures(object = obj))
obj <- RunUMAP(obj, dims = 1:32)
obj

In [21]:
SaveH5Seurat(obj, 'path/result/comparison/dcm_hcm/midas_embed/raw_data/obj.h5seurat', overwrite = T)

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA



In [ ]:
obj